In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, BatchNormalization, Conv2D, MaxPooling2D, Dropout
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import os
from PIL import Image

2023-04-03 10:33:43.310140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 10:33:48.256629: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class BirdDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, use_rows, batch_size, csv_file="birds.csv"):
        self.csv_df = pd.read_csv(csv_file)
        self.use_rows = use_rows
        self.batch_size = batch_size 
        self.dataset = pd.DataFrame(self.csv_df).to_numpy()
    
        self.labels = self.dataset[:,2]
        self.img_filepaths = self.dataset[:,1]
        self.class_ids = self.dataset[:,0]
        
        (x,) = self.img_filepaths.shape
        print(self.dataset.shape)
        print(self.img_filepaths[0])
        image = Image.open(self.img_filepaths[0])
        image_array = np.expand_dims(np.asarray(image), axis=0)
        
#         print(image_array)
        print(image_array.shape)
        
#         self.images = []
        
#         for i in range(0, 10):
#             image = Image.open(self.img_filepaths[0])
#             image_array = np.expand_dims(np.asarray(image), axis=0)
#             self.images.append(image_array)
            
#         self.images = np.array(self.images)
        
#         print(self.images.shape)
    
    def __len__(self):
        # batches_per_epoch is the total number of batches used for one epoch
        batches_per_epoch = int(len(self.use_rows) / self.batch_size)
        return batches_per_epoch
    
    def __getitem__(self, index):
        # index is the index of the batch to be retrieved
        batch_ids = self.use_rows[index * self.batch_size: (index + 1) * self.batch_size]
        
        x = None
        y = None
        
#         print("batch index: "+str(index))
#         print("batch ids: " + str(batch_ids))
        
        for curr_id in batch_ids:
            image = Image.open(self.img_filepaths[curr_id]).resize((224,224))
#             image = image.convert("L")
#             np_row = np.asarray(image)
            image_array = np.expand_dims(np.asarray(image),axis=0)
            
            if x is None:
                x = image_array
                y = self.class_ids[curr_id]
            else:
                x = np.vstack((x, image_array))
                y = np.vstack((y, self.class_ids[curr_id]))
                
    
#             print(self.class_ids[curr_id-1])
            
        
#         images = np.array(images)
#         id_labels = np.array(id_labels)
#         print(id_labels)
#         print(images.shape)
#         print(id_labels.shape)
#         print(self.class_ids)
        
#         print(self.class_ids.shape)
        
        
        y = tf.keras.utils.to_categorical(y, num_classes=511)
#         print(x.shape)
#         print(y.shape)
        return x, y            
            
#     def on_epoch_end(self):
#         np.random.shuffle(self.split_ids)
            
            

In [3]:
use_rows = list(range(0,87050))
bird_data_generator = BirdDataGenerator(csv_file="birds.csv", use_rows=use_rows, batch_size=128)
bird_data_generator.__getitem__(0)

(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


(array([[[[180, 172, 149],
          [186, 178, 155],
          [186, 180, 158],
          ...,
          [181, 180, 198],
          [179, 180, 200],
          [178, 179, 199]],
 
         [[182, 174, 153],
          [190, 182, 161],
          [190, 184, 162],
          ...,
          [182, 181, 199],
          [176, 177, 197],
          [178, 179, 199]],
 
         [[186, 178, 159],
          [195, 187, 168],
          [196, 189, 170],
          ...,
          [184, 183, 199],
          [180, 179, 197],
          [183, 182, 200]],
 
         ...,
 
         [[151, 138, 121],
          [172, 159, 142],
          [178, 165, 146],
          ...,
          [189, 190, 211],
          [191, 189, 210],
          [169, 167, 188]],
 
         [[156, 140, 124],
          [155, 139, 123],
          [161, 148, 131],
          ...,
          [186, 188, 211],
          [182, 185, 204],
          [168, 171, 188]],
 
         [[157, 141, 125],
          [156, 140, 124],
          [167, 154, 137],
   

In [4]:
def bird_cnn_model(csv_file):
    model = Sequential([
        InputLayer(input_shape=(224,224,3)),
        Conv2D(filters=32, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=2),
        Flatten(),
        Dense(511, activation="softmax")
    ])
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])
    
    print(model.summary())
    train_rows = list(range(0,81950))
    train_generator = BirdDataGenerator(csv_file=csv_file, use_rows=train_rows, batch_size=64)
    h = model.fit(x=train_generator, epochs=10, verbose=1)
    print(h)
    
#     validation_rows = list(range(84500, 87050))
#     validation_generator = BirdDataGenerator(csv_file=csv_file, use_rows=validation_rows, batch_size=128)
    
    test_rows = list(range(81950, 84500))
    test_generator = BirdDataGenerator(csv_file=csv_file, use_rows=test_rows, batch_size=128)
    model.evaluate(x=test_generator)
    
    return model
    

In [5]:
cnn_model = bird_cnn_model("birds.csv")

2023-04-02 14:31:16.564058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:31:16.652709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:31:16.653512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:31:16.657730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:31:16.658466: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 511)               201473503 
                                                                 
Total params: 201,474,527
Trainable params: 201,474,463


2023-04-02 14:31:18.741696: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-02 14:31:20.550433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-02 14:31:21.391033: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-02 14:31:21.805081: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fb198ba34f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-02 14:31:21.805133: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5
2023-04-02 14:31:21.961604: I tensorflow/tsl/p

   1/1280 [..............................] - ETA: 1:20:56 - loss: 8.1817 - accuracy: 0.0000e+00

2023-04-02 14:31:22.434062: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.49GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-02 14:31:22.434215: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.49GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1280/1280 [==============================] - 580s 451ms/step - loss: 6.8490 - accuracy: 0.0266
Epoch 2/10
1280/1280 [==============================] - 574s 448ms/step - loss: 3.6600 - accuracy: 0.2914
Epoch 3/10
1280/1280 [==============================] - 557s 435ms/step - loss: 1.8589 - accuracy: 0.6663
Epoch 4/10
1280/1280 [==============================] - 573s 447ms/step - loss: 0.8545 - accuracy: 0.8751
Epoch 5/10
1280/1280 [==============================] - 572s 447ms/step - loss: 0.4125 - accuracy: 0.9534
Epoch 6/10
1280/1280 [==============================] - 562s 439ms/step - loss: 0.2157 - accuracy: 0.9818
Epoch 7/10
1280/1280 [==============================] - 570s 445ms/step - loss: 0.1241 - accuracy: 0.9920
Epoch 8/10
1280/1280 [==============================] - 586s 457ms/step - loss: 0.0783 - accuracy: 0.9964
Epoch 9/10
1280/1280 [==============================] - 569s 444ms/step - loss: 0.0533 - accuracy: 0.9982
Epoch 10/10
1280/1280 [==============================] - 

2023-04-02 16:06:33.583995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-02 16:06:35.739937: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 665.74MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-02 16:06:35.798953: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.90GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 1/19 [>.............................] - ETA: 44s - loss: 5.2174 - accuracy: 0.1641

2023-04-02 16:06:36.002345: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 665.74MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-02 16:06:36.002725: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


19/19 [==============================] - 21s 1s/step - loss: 4.4298 - accuracy: 0.2808


In [6]:
cnn_model.save("batch_normalized_10_epochs")

2023-04-02 16:07:09.610715: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 805891968 exceeds 10% of free system memory.
2023-04-02 16:07:10.335875: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 805891968 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: batch_normalized_10_epochs/assets


INFO:tensorflow:Assets written to: batch_normalized_10_epochs/assets


In [14]:
test_image = Image.open("images to predict/1.jpg").resize((224,224))
image_array = np.expand_dims(np.asarray(test_image),axis=0)
print(image_array)
cnn_model.predict(image_array)

[[[[ 87  79  42]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 87  76  54]
   [ 94  84  57]
   [ 95  86  57]]

  [[ 88  80  43]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 89  78  56]
   [ 96  86  59]
   [ 94  85  56]]

  [[ 88  80  43]
   [ 89  81  44]
   [ 90  82  46]
   ...
   [ 96  86  61]
   [ 90  78  52]
   [101  89  63]]

  ...

  [[103  92  74]
   [105  94  76]
   [106  94  78]
   ...
   [ 99  86  54]
   [ 97  83  54]
   [ 97  84  52]]

  [[104  92  76]
   [106  94  78]
   [107  95  79]
   ...
   [ 99  86  52]
   [ 96  83  51]
   [ 95  82  48]]

  [[106  94  78]
   [108  96  80]
   [109  97  81]
   ...
   [100  87  53]
   [ 97  84  50]
   [ 96  83  49]]]]
1/1 [==============================] - 0s 28ms/step


array([[2.57067131e-06, 3.99548007e-05, 3.84753780e-03, 1.49096865e-02,
        1.58080012e-02, 5.90787444e-04, 1.26173254e-04, 1.12543837e-03,
        5.93369012e-04, 2.58353393e-04, 8.75800833e-06, 1.16440999e-04,
        3.47768539e-04, 9.35269236e-06, 5.31022029e-04, 3.08653180e-05,
        1.29992513e-05, 2.53940798e-05, 4.21276374e-04, 5.12978341e-03,
        4.32076013e-06, 2.98589410e-04, 3.36751873e-05, 1.32276805e-03,
        1.09392253e-03, 5.11953346e-02, 3.47930938e-04, 2.87048920e-06,
        6.33914350e-03, 4.41652592e-05, 8.55482882e-04, 4.23731096e-03,
        1.99401602e-05, 8.74998950e-05, 5.67223004e-04, 6.12738775e-04,
        3.15797915e-05, 7.62827112e-05, 1.08834956e-06, 3.13567871e-04,
        2.22418574e-03, 6.42655592e-04, 1.77691909e-04, 3.49773956e-03,
        2.67815194e-05, 7.40081945e-04, 1.81456140e-04, 1.90441089e-04,
        5.54084618e-05, 9.02732645e-05, 7.87902437e-03, 1.55617110e-02,
        1.59787363e-04, 2.21194816e-03, 6.60426595e-05, 6.289677

In [3]:
one_epoch_model = tf.keras.models.load_model("batch_normalized_1_epoch")
one_epoch_model.summary()

2023-04-03 10:33:53.430591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.065585: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.066989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.084907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.085734: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 511)               201473503 
                                                                 
Total params: 201,474,527
Trainable params: 201,474,463


In [9]:
test_image = Image.open("images to predict/1.jpg").resize((224,224))
image_array = np.expand_dims(np.asarray(test_image),axis=0)
print(image_array)
predict_x = one_epoch_model.predict(image_array)
classes_x = np.argmax(predict_x, axis=1)
print(classes_x)

[[[[ 87  79  42]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 87  76  54]
   [ 94  84  57]
   [ 95  86  57]]

  [[ 88  80  43]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 89  78  56]
   [ 96  86  59]
   [ 94  85  56]]

  [[ 88  80  43]
   [ 89  81  44]
   [ 90  82  46]
   ...
   [ 96  86  61]
   [ 90  78  52]
   [101  89  63]]

  ...

  [[103  92  74]
   [105  94  76]
   [106  94  78]
   ...
   [ 99  86  54]
   [ 97  83  54]
   [ 97  84  52]]

  [[104  92  76]
   [106  94  78]
   [107  95  79]
   ...
   [ 99  86  52]
   [ 96  83  51]
   [ 95  82  48]]

  [[106  94  78]
   [108  96  80]
   [109  97  81]
   ...
   [100  87  53]
   [ 97  84  50]
   [ 96  83  49]]]]
1/1 [==============================] - 0s 47ms/step
[507]


In [15]:
# Generate class labels
csv_df = pd.read_csv("birds.csv")
dataset = pd.DataFrame(csv_df).to_numpy()
labels = dataset[:,2]
class_ids = dataset[:,0]

print(labels.shape)
print(class_ids.shape)

label_mapping = {}
id_to_name = dict(zip(labels, class_ids))
print(id_to_name)

(87050,)
(87050,)
{'ABBOTTS BABBLER': 0.0, 'ABBOTTS BOOBY': 1.0, 'ABYSSINIAN GROUND HORNBILL': 2.0, 'AFRICAN CROWNED CRANE': 3.0, 'AFRICAN EMERALD CUCKOO': 4.0, 'AFRICAN FIREFINCH': 5.0, 'AFRICAN OYSTER CATCHER': 6.0, 'AFRICAN PIED HORNBILL': 7.0, 'AFRICAN PYGMY GOOSE': 8.0, 'ALBATROSS': 9.0, 'ALBERTS TOWHEE': 10.0, 'ALEXANDRINE PARAKEET': 11.0, 'ALPINE CHOUGH': 12.0, 'ALTAMIRA YELLOWTHROAT': 13.0, 'AMERICAN AVOCET': 14.0, 'AMERICAN BITTERN': 15.0, 'AMERICAN COOT': 16.0, 'AMERICAN FLAMINGO': 17.0, 'AMERICAN GOLDFINCH': 18.0, 'AMERICAN KESTREL': 19.0, 'AMERICAN PIPIT': 20.0, 'AMERICAN REDSTART': 21.0, 'AMERICAN ROBIN': 22.0, 'AMERICAN WIGEON': 23.0, 'AMETHYST WOODSTAR': 24.0, 'ANDEAN GOOSE': 25.0, 'ANDEAN LAPWING': 26.0, 'ANDEAN SISKIN': 27.0, 'ANHINGA': 28.0, 'ANIANIAU': 29.0, 'ANNAS HUMMINGBIRD': 30.0, 'ANTBIRD': 31.0, 'ANTILLEAN EUPHONIA': 32.0, 'APAPANE': 33.0, 'APOSTLEBIRD': 34.0, 'ARARIPE MANAKIN': 35.0, 'ASHY STORM PETREL': 36.0, 'ASHY THRUSHBIRD': 37.0, 'ASIAN CRESTED IBIS': 38.